# 📈 Stock Price Prediction with SQL and Python

This notebook walks you through predicting stock prices using historical data stored in an SQLite database.


In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("stocks.db")
cursor = conn.cursor()

# Read SQL script and execute it
with open("stock_data.sql", "r") as f:
    cursor.executescript(f.read())
conn.commit()


In [ ]:
# Load stock data from SQLite into a DataFrame
df = pd.read_sql_query("SELECT * FROM stocks", conn)
df['date'] = pd.to_datetime(df['date'])  # Convert date column
df.sort_values("date", inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# Plot stock closing prices
plt.figure(figsize=(8,4))
plt.plot(df['date'], df['close'], marker='o')
plt.title("Stock Closing Prices")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.grid(True)
plt.show()


In [ ]:
# Use day index as feature to predict closing price
df['day_index'] = range(len(df))

X = df[['day_index']]
y = df['close']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")


In [ ]:
# Visualize predictions
plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.plot(X_test, y_pred, color='red', label='Predicted')
plt.title("Actual vs Predicted Closing Prices")
plt.xlabel("Day Index")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()
